In [27]:
%pip install requests beautifulsoup4 openai

Note: you may need to restart the kernel to use updated packages.


In [28]:
import requests
from bs4 import BeautifulSoup
import time
import logging
from openai import OpenAI

In [29]:
# Set OpenAI and SERP API keys

openai_api_key = 'sk-proj-lXv4D7Ep2mVnjng7i58sT3BlbkFJDm0BMQKrCn2mv56VLs1S'
serp_api_key = 'c5447badf6cccf8421758fa2969175d1c32f08368b47fa4a62cd340d6525d40e'

In [30]:
# Initialize the OpenAI client with API key
client = OpenAI(api_key=openai_api_key)

In [31]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,  # Capture INFO level and above
    format='%(asctime)s - %(levelname)s - %(message)s'  # Simple format with timestamp, log level, and message
)

In [32]:
# Define scraping function to extract content for a given URL
def scrape_targeted_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract basic content
        #title = soup.title.string if soup.title else ''
        #meta_description = soup.find('meta', attrs={'name': 'description'})
        #meta_description = meta_description['content'] if meta_description else ''
        body_text = ' '.join(p.get_text() for p in soup.find_all('p'))
        
        return {
            #"title": title,
            #"meta_description": meta_description,
            "body_text": body_text
        }
    except Exception as e:
        logging.error(f"Error scraping {url}: {e}")
        return {}

In [33]:
# Define Search function using SERP API

def search_company(company_name):
    search_url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": company_name,
        "api_key": serp_api_key  # Use the SERP API key here
    }
    response = requests.get(search_url, params=params)
    
    if response.status_code == 200:
        search_results = response.json()
        results = []
        for result in search_results.get('organic_results', [])[:3]:  # Limit to top 3 results
            results.append({
                'url': result.get('link'),
                'text': result.get('snippet', '')
            })
        return results
    else:
        logging.error(f"Request failed with status code: {response.status_code}")
        return None

In [34]:
# Define a function to summarize search results' content extracted from scraping using Open AI

def summarize_search_results(search_results):
    all_texts = []
    for result in search_results:
        content = scrape_targeted_content(result['url'])
        text = content.get("body_text", "")
        if text:
            all_texts.append(text)
        time.sleep(1)  # Respectful delay between requests
    
    combined_text = ' '.join(all_texts)
    max_length = 1000  # Maximum token limit for GPT-4 is around 4096 tokens
    if len(combined_text) > max_length:
        combined_text = combined_text[:max_length]
    
    print(f"Combined text for summarization:\n{combined_text[:1000]}...")  # Print only the first 500 characters for brevity
    
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Summarize the following text: {combined_text}"}
        ],
        model="gpt-4",
        max_tokens=300,
        temperature=0.7
    )
    
    #summary = response['choices'][0]['message']['content'].strip()
    summary = response.choices[0].message.content
    return summary

In [35]:
# Combine the functions to create an AI sales agent

def ai_sales_agent(company_name):
    search_results = search_company(company_name)
    if search_results:
        for result in search_results:
            content = scrape_targeted_content(result['url'])
            #print(f"URL: {result['url']}\nTitle: {content['title']}\nMeta Description: {content['meta_description']}\nBody Text: {content['body_text'][:200]}...")  # Print only the first 200 characters of the body text
        
        summary = summarize_search_results(search_results)
        results_output = "\n\n".join([f"URL: {result['url']}\nText: {result['text']}" for result in search_results])
        return f"Summary for {company_name}:\n{summary}\n\nURLs identified:\n{results_output}"
    else:
        return "No results found for the company name."

In [39]:
# Run the AI sales agent for a given company name

company_name = "Shell"  # Replace with the company name you want to search for
summary = ai_sales_agent(company_name)
print(f"Summary for {company_name}:\n{summary}")

Combined text for summarization:
We’re finding ways to bring energy to more people in more ways every day, so that all of us can be a part of the changing energy system. Because Powering Progress means providing more and cleaner energy across the country.   Read more Read more Read more Read more See how! Read the full article Read the full article Read the full article Read the full article Read the full article Watch the video! Read the full article Read the full article Read the full article Read the full article Read the full article Read past articles, press releases, and announcements.    Gretchen Watkins, President, Shell USA, Inc.  We’re finding ways to bring energy to more people in more ways every day, so that all of us can be a part of the changing energy system. Because Powering Progress means providing more and cleaner energy across the country.   Get the most out of driving. Discover the wide range of products Shell offers to improve your driving experience, and programs 

2024-07-17 23:03:50,307 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Summary for Shell:
Summary for Shell:
The text discusses efforts to distribute energy to more people in diverse ways daily, thus allowing everyone to participate in the evolving energy system. Powering Progress aims to provide increased and cleaner energy nationwide. Shell USA's President, Gretchen Watkins, also emphasizes this theme. Furthermore, Shell offers a variety of products and programs to enhance the driving experience and save money while doing so.

URLs identified:
URL: https://www.shell.com/
Text: Shell's award-winning digital stories channel. Our team of writers and reporters offer fresh insights into energy, technology and the people and ideas powering ...

URL: https://www.shell.us/
Text: Shell in the United States explores and produces energy products - fuels, oil, natural gas, lubricants, LPG, chemicals; with major projects in the Gulf of ...

URL: https://en.wikipedia.org/wiki/Shell_plc
Text: Shell plc is a British multinational oil and gas company headquartered in Lo